# Movie Genre Prediction by Poster
The goal of this notebook is to try to predict a movie's genre by looking at the poster it has.
To achieve this goal I will:
1. Use [The Movies Dataset](https://www.kaggle.com/rounakbanik/the-movies-dataset) to get 45k movie titles.
2. Use the [omdb_api](http://www.omdbapi.com/) to get the posters for the movies and store them localy.
3. Use Tensorflow and Keras to create a ConvNet that will classify the images.

## Project Structure
1. Data loading, acquisition and cleaning
2. Model creation.
3. Model evaluation.

In [1]:
# Library Loading
import json
from urllib.parse import urlencode
import urllib.request
import requests
import pandas as pd
import os

# Loading the 
OMDB_KEY = json.loads(open('apikeys/apikey.json').read())['key']

## 1. Data loading, acquisition and cleaning

### 1.1 Loading 'The Movies' dataset.

In [2]:
the_movies = pd.read_csv('./datasets/The_Movies/movies_metadata.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 1.2 Defining the API functions

In [3]:
def get_movie(imdbid):
    '''Gets the movie by imdb id and returns json with the title, genre and imdb id.
       Input args:
       - imdbid: the imdb id of the character
       Returns:
       - If the API responds succesfully return the genres, otherwise return NA
    - 
    '''
    url = 'http://www.omdbapi.com/?apikey=' + OMDB_KEY + '&i=' + str(imdbid)
    r = requests.get(url)
    if r.status_code == 200:
        movie_json = r.json()
        return movie_json['Genre']
    else:
        return 'NA'

def get_poster(imdb_id, path, genre, dataset_type, image_heigth, item_id):
    '''Gets the movie poster as a jpg and saves it to the path.
       Input args:
       - imbd_id: the imdb id of the movie,
       - path: where the dataset will be stored,
       - genre: the genre of the movie,
       - dataset_type: whether we are creating the training or testing,
       - image_height: the height of the poster - api query
       - item_id: identifier to attach to the filename
       Returns:
       - None
    '''
    # construct the requests url
    url = 'http://img.omdbapi.com/?apikey=' + OMDB_KEY + '&i=' + str(imdb_id) + '&h=' + str(image_heigth)
    # create the dir where we store the posters based on the genre and the dataset type
    path = os.path.join(os.getcwd(), path, dataset_type, genre + '/')
    # check if the folder is already_created
    if not os.path.exists(path):
        os.makedirs(path)
    r = requests.get(url)
    filename = os.path.join(os.getcwd(), path, genre + '.' + str(item_id) + '.jpg')
    if r.status_code == 200:
        with open(filename, 'wb') as w:
            w.write(r.content)

### 1.3 Downloading the movie posters.

As a movie can have multiple genres I will make an assumption here - if a genre is listed first - that will be the genre of the movie. Let's create a function that will get only the main genre of the movie from the dataset. If it doesn't find one - download it straight from imdb.

In [4]:
import ast

def get_main_genre(dataset_genre, imdb_id):
    '''
        Gets the main genre of the movie. If there is none listed - pull one directly from imdb.
        Input args:
        - genre_row: the dataset value of the genres for the movie,
        - imdb_id: the imdb_id of the movie
        Returns:
        - main_genre: the main genre of the movie 
    '''
    dataset_genre = ast.literal_eval(dataset_genre)
    if len(dataset_genre) == 0:
        try: 
            main_genre = get_movie(imdb_id).replace(' ', '').split(',')[0]
        except KeyError:
            main_genre = 'NA'
    else:
        main_genre = dataset_genre[0]['name']
        if main_genre == 'N/A':
            main_genre = 'NA'
    return main_genre

In [157]:
# DANGER: Slow Code!
the_movies['main_genre'] = the_movies.apply(lambda x: get_main_genre(x['genres'], x['imdb_id']), axis=1)

In [158]:
the_movies['main_genre'].unique().shape 

(31,)

In [189]:
# quick fix for N/As
the_movies[the_movies['main_genre'] == 'N/A'] = 'NA'

All in all we got 31 classes. Let's create the train - test split now, as we will need it for movie poster download.

In [191]:
from sklearn.model_selection import train_test_split

movies_train, movies_test = train_test_split(the_movies, test_size=0.3, random_state=0)

Finally downloading the movies

In [ ]:
# downloading the posters for the train set
for idx, row in movies_train.iterrows():
    # get_poster(imdb_id, path, genre, dateset_type, image_heigth, item_id):
    get_poster(row['imdb_id'], 'datasets/The_Movies/posters', row['main_genre'], 'train', 600, idx)
# downloading the posters for the test set
for idx, row in movies_test.iterrows():
    get_poster(row['imdb_id'], 'datasets/The_Movies/posters', row['main_genre'], 'test', 600, idx)

The code above has been moved to a script so it can stay overnight and download all of the movies' posters. 
Since that has been done lets move on to what genres we have actually downloaded.

In [7]:
os.listdir('/media/fury/data/Scripts/the_movies_data_scraper/datasets/The_Movies/posters/train')

['Sci-Fi',
 'Fantasy',
 'Drama',
 'Biography',
 'Animation',
 'Science Fiction',
 'Odyssey Media',
 'Romance',
 'Crime',
 'Adventure',
 'History',
 'Aniplex',
 'War',
 'Carousel Productions',
 'Foreign',
 'Documentary',
 'Thriller',
 'Music',
 'Action',
 'Adult',
 'Western',
 'Short',
 'Family',
 'Horror',
 'Comedy',
 'NA',
 'Mystery',
 'Musical',
 'TV Movie']

From the list, displayed above we can see somethings we can edit/remove. For example 
- Sci-Fi and Science Fiction are the same thing so both dirs can be merged to a common category 'Sci-Fi'
- 'Aniplex', 'Odyssey Media' and 'Carousel Productions' are actually names of companies. Will see how much samples they contribute and if it is not significant (~100 or more)they will be deleted.
- NA - The same treatment - check the number of samples and delete them.

#### Results
- Sci-Fi - it seems that there are only 5 posters in Sci-Fi - thus I can merge it into 'Science Fiction'
- 'Odyssey Media' and 'Carousel Productions' and 'NA' contained no information whatsover.
- 'Removed also 'Adult' as it contained no posters.

#### Aditional Checks performed
- Whether both dirs contain the same genres and the same number of genres.

## 2. Model creation.

### 2.1 Importing the neccesarry libraries.

In [3]:
# Importing the libraries
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Using TensorFlow backend.


### 2.2  Loading the data.
With the help of Keras that should be quite straightforward.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_location = '/media/fury/data/Scripts/the_movies_data_scraper/datasets/The_Movies/posters/train'
test_location = '/media/fury/data/Scripts/the_movies_data_scraper/datasets/The_Movies/posters/test'

train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# The generators
train_generator = train_datagen.flow_from_directory(
    train_location,
    target_size = (300, 300),
    color_mode = 'rgb',
    batch_size = 32,
    class_mode='categorical',
    seed=42
)

test_generator = test_datagen.flow_from_directory(
    test_location.
    target_size=(300)
)

